In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [13]:
import os
print(os.getcwd())

/Volumes/SSD 1TB/6th Sem/Individual Project/Linguadash/experiments


In [15]:
df = pd.read_csv("../data/raw/english_language_learning_1M.csv")

print("Dataset shape:", df.shape)
df.head()

Dataset shape: (1000000, 37)


,content_id,user_level,topic,content_type,difficulty,game_mode,error_category,incorrect_text,correct_text,error_type,...,attempts_avg,hint_usage_rate,completion_rate,skip_rate,replay_rate,retention_score,improvement_score,created_date,recommended_next,skill_tags
0,ENG0000001,B1_Intermediate,Travel,sentence,Easy,Speed_Round,grammar,Everyone have finished their work.,Everyone has finished their work.,grammar,...,1.3,0.15,0.79,0.20,0.09,0.75,0.50,2024-07-17,Same_Error_Type,grammar;Travel
1,ENG0000002,C1_Advanced,Business,short_paragraph,Easy,Error_Hunt,vocabulary,The weather is too hot. I can't bare it.,The weather is too hot. I can't bear it.,vocabulary,...,2.0,0.66,0.89,0.05,0.25,0.68,0.71,2023-06-18,Harder_Difficulty,vocabulary;Business;Word_Power
2,ENG0000003,B2_Upper_Intermediate,Technology,sentence,Easy,Quick_Fix,punctuation,"The movie was boring, I left early.","The movie was boring, so I left early.",punctuation,...,2.2,0.32,0.81,0.13,0.30,0.81,0.54,2023-03-15,Similar_Topic,punctuation;Technology
3,ENG0000004,B1_Intermediate,Business,sentence,Medium,Boss_Battle,sentence_structure,She speaks fluently English.,She speaks English fluently.,sentence_structure,...,2.5,0.11,0.67,0.29,0.18,0.65,0.72,2025-05-03,Same_Error_Type,sentence_structure;Business
4,ENG0000005,A2_Elementary,Technology,sentence,Hard,Error_Hunt,punctuation,"She is talented, she works hard too.","She is talented, and she works hard too.",punctuation,...,2.4,0.63,0.63,0.32,0.10,0.89,0.64,2023-11-23,Mixed_Practice,punctuation;Technology;Advanced_Grammar


In [16]:
df = df[
    [
        "incorrect_text",
        "correct_text",
        "error_category",
        "difficulty",
        "user_level"
    ]
]

df.head()

,incorrect_text,correct_text,error_category,difficulty,user_level
0,Everyone have finished their work.,Everyone has finished their work.,grammar,Easy,B1_Intermediate
1,The weather is too hot. I can't bare it.,The weather is too hot. I can't bear it.,vocabulary,Easy,C1_Advanced
2,"The movie was boring, I left early.","The movie was boring, so I left early.",punctuation,Easy,B2_Upper_Intermediate
3,She speaks fluently English.,She speaks English fluently.,sentence_structure,Medium,B1_Intermediate
4,"She is talented, she works hard too.","She is talented, and she works hard too.",punctuation,Hard,A2_Elementary


In [17]:
# Remove missing values
df = df.dropna()

# Convert text to lowercase
df["incorrect_text"] = df["incorrect_text"].str.lower()
df["correct_text"] = df["correct_text"].str.lower()

print("After cleaning:", df.shape)

After cleaning: (1000000, 5)


In [18]:
label_encoder = LabelEncoder()
df["error_label"] = label_encoder.fit_transform(df["error_category"])

# Check mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
label_mapping

{'grammar': np.int64(0),
 'punctuation': np.int64(1),
 'sentence_structure': np.int64(2),
 'spelling': np.int64(3),
 'vocabulary': np.int64(4)}

In [19]:
X = df["incorrect_text"]
y = df["error_label"]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))

Training samples: 800000
Testing samples: 200000


In [21]:
tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),
    stop_words="english"
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [22]:
classifier = LogisticRegression(
    max_iter=1000,
    n_jobs=-1
)

classifier.fit(X_train_tfidf, y_train)

/Volumes/SSD 1TB/6th Sem/Individual Project/Linguadash/venv/lib/python3.14/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [23]:
y_pred = classifier.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 1.0

Classification Report:

                    precision    recall  f1-score   support

           grammar       1.00      1.00      1.00     80088
       punctuation       1.00      1.00      1.00     29881
sentence_structure       1.00      1.00      1.00     19981
          spelling       1.00      1.00      1.00     29971
        vocabulary       1.00      1.00      1.00     40079

          accuracy                           1.00    200000
         macro avg       1.00      1.00      1.00    200000
      weighted avg       1.00      1.00      1.00    200000



In [24]:
def predict_error_category(sentence):
    sentence = sentence.lower()
    vector = tfidf.transform([sentence])
    pred_label = classifier.predict(vector)[0]
    return label_encoder.inverse_transform([pred_label])[0]


test_sentence = "He go to school yesterday"
print("Prediction:", predict_error_category(test_sentence))

Prediction: grammar


In [25]:
import joblib

joblib.dump(classifier, "error_classifier.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']